# Richmond Police Activity Maps

The maps below are obtained from Richmond Police Department and Wayne County Sheriff media reports.

In [194]:
from __future__ import division, print_function
import pandas as pd
import os
import numpy as np
import folium
from folium import plugins
import datetime
import calendar


def read_input_data(include=None, exclude=None):
    '''Read the input data'''
    df = pd.read_csv('reports_all.csv')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Latitude', 'Longitude'])
    if include:
        df = df[df.Details.str.contains('|'.join(include), case=False, na=False)]
    if exclude:
        df = df[~df.Details.str.contains('|'.join(exclude), case=False, na=False)]
    return df


def add_dots(m, df):
    '''Add dots to the map'''
    for index, row in df.iterrows():
        if row['department'] == 'richmond':
            color = 'red'
        else:
            color = 'blue'
        details = ''
        if str(row['Date']) is not None:
            details += '<b>' + str(row['Date']) + '</b><br>\n'
        if row['Address'] is not None:
            details += '<i>' + row['Address'].replace(', Richmond, IN  47374', '') \
                                             .replace(', Wayne County, IN', '') + '</i><br>'
        if row['Details'] is not None:
               details += str(row['Details']).replace("'", "\\'") + '\n'
        folium.Circle(location=[row['Latitude'], row['Longitude']],
                      popup=details,
                      radius=15,
                      color=color,
                      fill_color=color
                      ).add_to(m)

        
def create_map(include=None, exclude=None, dots=True, static_heat_map=True, time_lapse_heat_map=False):
    '''Add overall results to the map'''

    df = read_input_data(include=include, exclude=exclude)
    
    m = folium.Map(location=[39.8289, -84.8902],
                   tiles='Stamen Toner',
                   zoom_start=14,
                   detect_retina=True,
                   prefer_canvas=True)

    if dots:
        add_dots(m, df)
        
    if static_heat_map:
        heat_map_array = df[['Latitude', 'Longitude']].values
        m.add_child(plugins.HeatMap(heat_map_array))
        
    if time_lapse_heat_map:
        df['Month'] = df['Date'].map(lambda x: x.month)
        min_month = min(df['Month'])
        max_month = max(df['Month'])
        heat_map_time_array = []
        months = []
        for month in range(min_month, max_month+1):
            df_month = df[df['Month'] == month]
            heat_map_time_array.append(df_month[['Latitude', 'Longitude']].values.tolist())
            months.append(calendar.month_name[month])
        m.add_child(plugins.HeatMapWithTime(heat_map_time_array, index=months))
    return m

In [195]:
create_map()

Figure 1. Police activity map from the Richmond Police Department (red dots) and the Wayne County Sheriff's office (blue dots). More information about each event can be obtained by clicking on a dot.

In [196]:
create_map(dots=False, static_heat_map=False, time_lapse_heat_map=True)

Figure 2. Police activity map from the Richmond Police Department and the Wayne County Sheriff's office for each month within the selected year. Use the navigation bar above to view results from different months.

In [197]:
include = ['controlled substance', 'overdose', 'narcotic', 'drug', 'meth', 'marijuana', 'heroin', 'legend', 'cocaine'
           'legend', 'paraphernalia', 'syringe', 'possession', 'common nuisance']
create_map(include=include)

In [ ]:
Figure 3. Drug and drug-related police activity.

In [198]:
include = ['paraphernalia', 'syringe', 'possession', 'common nuisance']
exclude = ['operating while intoxicated', 'theft', 'knife', 'controlled substance', 'handgun', 'overdose', 
           'narcotic', 'drug', 'meth', 'marijuana', 'heroin', 'legend', 'cocaine']
create_map(include=include, exclude=exclude)

Figure 4. Drug-related police activity where no actual drugs were involved.

In [199]:
include = ['burglary', 'theft', 'shoplifting', 'shop lifting', 'conversion', 'stolen', 'steal', 'stole']
create_map(include=include)

Figure 5. Theft-related police activity (activity where any of the follow words are mentioned: 'burglary', 'theft', 'shoplifting', 'shop lifting', 'conversion', 'stolen', 'steal', 'stole').